In [2]:
import pandas as pd
result = pd.read_pickle('result.pickle')
result

,着順,枠番,馬番,馬名,斤量,騎手,単勝,人気,性,齢,体重,体重増減
202001010101,1,6,6,ウインルーア,54.0,横山武史,16.0,3.0,牝,2,438,4
202001010101,2,2,2,アークライト,54.0,ルメール,1.9,2.0,牡,2,510,0
202001010101,3,3,3,ギャラントウォリア,54.0,池添謙一,1.8,1.0,牡,2,482,-6
202001010101,4,1,1,ジュンブーケ,52.0,亀田温心,22.2,4.0,牝,2,442,0
202001010101,5,4,4,キタノマンゲツ,54.0,藤岡康太,55.7,5.0,牡,2,426,-8
...,...,...,...,...,...,...,...,...,...,...,...,...
202010020812,11,1,1,キスラー,57.0,武豊,6.5,4.0,牡,4,476,-2
202010020812,12,8,14,ポンペイワーム,57.0,浜中俊,19.8,8.0,牡,4,522,-8
202010020812,13,8,15,アルマトップエンド,57.0,国分恭介,75.0,14.0,牡,5,510,0
202010020812,14,2,3,ノイーヴァ,55.0,太宰啓介,46.1,10.0,牝,4,488,-12


In [3]:
clip_rank = lambda x: x if x<4 else 4
result["rank"] = result["着順"].map(clip_rank)

In [4]:
result["rank"].value_counts()

4    33869
1     3128
3     3125
2     3117
Name: rank, dtype: int64

In [5]:
result.drop(["着順","馬名"],axis=1,inplace=True)

In [6]:
result_d = pd.get_dummies(result)

In [7]:
result_d

,枠番,馬番,斤量,単勝,人気,齢,体重,体重増減,rank,騎手_シュタル,...,騎手_鮫島克也,騎手_鮫島克駿,騎手_鮫島良太,騎手_鴨宮祥行,騎手_黒岩悠,騎手_黛弘人,騎手_Ｍ．デム,性_セ,性_牝,性_牡
202001010101,6,6,54.0,16.0,3.0,2,438,4,1,0,...,0,0,0,0,0,0,0,0,1,0
202001010101,2,2,54.0,1.9,2.0,2,510,0,2,0,...,0,0,0,0,0,0,0,0,0,1
202001010101,3,3,54.0,1.8,1.0,2,482,-6,3,0,...,0,0,0,0,0,0,0,0,0,1
202001010101,1,1,52.0,22.2,4.0,2,442,0,4,0,...,0,0,0,0,0,0,0,0,1,0
202001010101,4,4,54.0,55.7,5.0,2,426,-8,4,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202010020812,1,1,57.0,6.5,4.0,4,476,-2,4,0,...,0,0,0,0,0,0,0,0,0,1
202010020812,8,14,57.0,19.8,8.0,4,522,-8,4,0,...,0,0,0,0,0,0,0,0,0,1
202010020812,8,15,57.0,75.0,14.0,5,510,0,4,0,...,0,0,0,0,0,0,0,0,0,1
202010020812,2,3,55.0,46.1,10.0,4,488,-12,4,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# from sklearn.preproccessing import StandardScaler

X = result_d.drop(["rank"],axis=1)
y = result_d["rank"]
#stratifyはデータの出現の偏りを実データと同じ割合にするもの（rankに4が多いがちゃんと1,2,3も出現してもらうようにするため）


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [10]:
model = LogisticRegression()
model.fit(X_train, y_train)

C:\Users\owner\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [11]:
print(model.score(X_train, y_train), model.score(X_test, y_test)) # 一見正解率８割弱に見える

0.7832953381570688 0.7833024976873265


In [12]:
y_pred = model.predict(X_test)

In [13]:
pd.Series(y_pred).value_counts() #全て4と予想している（そもそもresult["rank"]は4が多いデータのため）→不均衡データといいそれに応じた処理が必要

4    12972
dtype: int64

In [19]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1:rank_1, 2:rank_2, 3:rank_3, 4:rank_1}, random_state=71)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [24]:
y_train_rus.value_counts()

4    2190
1    2190
3    2187
2    2182
Name: rank, dtype: int64

In [25]:
model = LogisticRegression()
model.fit(X_train_rus, y_train_rus)

print(model.score(X_train, y_train), model.score(X_test, y_test))

0.5522516271847226 0.5509559050262103


C:\Users\owner\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [26]:
y_pred = model.predict(X_test)

In [27]:
pred_df = pd.DataFrame({"pred":y_pred, "actual":y_test})

In [30]:
pred_df[pred_df["pred"]==1]['actual'].value_counts() #1着予想のうち実際1着のものがどれくらいか

4    1598
1     610
2     522
3     404
Name: actual, dtype: int64

In [31]:
pred_df[pred_df["pred"]==4]['actual'].value_counts()

4    6234
3     220
2     151
1     103
Name: actual, dtype: int64

In [35]:
#回帰係数
pd.Series(model.coef_[0],index=X.columns).sort_values()

人気        -0.140234
齢         -0.016299
枠番        -0.005109
単勝        -0.002834
騎手_和田竜二   -0.002682
             ...   
騎手_松山弘平    0.003236
騎手_福永祐一    0.004079
騎手_ルメール    0.004581
騎手_川田将雅    0.006364
馬番         0.006974
Length: 185, dtype: float64

In [36]:
result[result["騎手"]=='川田将雅']['rank'].value_counts()

4    227
1    156
2    103
3     58
Name: rank, dtype: int64

In [37]:
result[result["騎手"]=='和田竜二']['rank'].value_counts()

4    540
2     87
3     68
1     52
Name: rank, dtype: int64

In [40]:
result[result["人気"]==1]['rank'].value_counts()

4    1159
1    1003
2     557
3     386
Name: rank, dtype: int64